In [2]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import gspread_dataframe as gd
import pandas as pd

### Raw Cashback File

In [3]:
# Read input files
raw_october = pd.read_csv('../Cashback_Payments/Raw_October.csv')

### Cohorts Info

In [4]:
cohorts_info = pd.read_csv('Cohorts.csv')

In [5]:
cohorts_info.head(10)

,email,klarUserId,Cashback_type,Opt_in_Cashback_October,Full cohort,op-int?
0,albertorh1999@hotmail.com,a712e4ed-350d-46f7-bed8-c8aa84b6fbe0,2K,1,2K1,YES
1,ntryf8e@gmail.com,52e251db-ad35-4d18-8997-0a9a426dd7c5,2K_or_10P,0,2K_or_10P0,YES
2,interunerm@gmail.com,009dd382-1a8f-443f-bec8-a161365655bc,10P,0,10P0,NO
3,oz11113333@gmail.com,8accbc3a-07d4-4ee2-97f2-008b930c658b,2K,1,2K1,YES
4,ruizlopezdavid@gmail.com,9ff30ca5-b210-4d0c-b677-84db3008b50e,2K_or_10P,0,2K_or_10P0,YES
5,jr10.angel@gmail.com,8adc19e4-92b3-435c-ac31-6edc209f5203,2K_or_10P,0,2K_or_10P0,YES
6,yhwh89rfc@gmail.com,be9012f7-0a41-4e37-aa21-d5439634d142,2K,1,2K1,YES
7,e.ribera26@gmail.com,3ecd1fcd-679c-4e2c-99eb-87e9c2487101,2K,1,2K1,YES
8,rigomercado1954@gmail.com,73e733ee-2a7b-4201-820d-6234959a2bc6,10P,0,10P0,NO
9,reyes.diana@tec.mx,d75c71aa-3a59-4eba-916b-2caaa6868533,10P,0,10P0,NO


In [6]:
# Delete users without user_id
cohorts_info = cohorts_info[~cohorts_info.klarUserId.isna()]
cohorts_info[cohorts_info.klarUserId.duplicated()]

,email,klarUserId,Cashback_type,Opt_in_Cashback_October,Full cohort,op-int?


In [7]:
cohorts_info.columns

Index(['email', 'klarUserId', 'Cashback_type', 'Opt_in_Cashback_October',
       'Full cohort', 'op-int?'],
      dtype='object')

In [8]:
# Needed columns
cohorts_info_clean = cohorts_info[['email','klarUserId', 'Cashback_type', 'op-int?']]
# Rename columns
cohorts_info_clean.columns = ['email','user_id', 'cashback_type', 'needed_opt_in']

In [9]:
# Remap op-int column
cohorts_info_clean.needed_opt_in = cohorts_info_clean.needed_opt_in.map({'YES':True, 'NO':False})

/var/folders/b_/lql2bh7s2797fh2q6lw2ttdh0000gn/T/ipykernel_29786/942361096.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cohorts_info_clean.needed_opt_in = cohorts_info_clean.needed_opt_in.map({'YES':True, 'NO':False})


In [10]:
cohorts_info_clean

,email,user_id,cashback_type,needed_opt_in
0,albertorh1999@hotmail.com,a712e4ed-350d-46f7-bed8-c8aa84b6fbe0,2K,True
1,ntryf8e@gmail.com,52e251db-ad35-4d18-8997-0a9a426dd7c5,2K_or_10P,True
2,interunerm@gmail.com,009dd382-1a8f-443f-bec8-a161365655bc,10P,False
3,oz11113333@gmail.com,8accbc3a-07d4-4ee2-97f2-008b930c658b,2K,True
4,ruizlopezdavid@gmail.com,9ff30ca5-b210-4d0c-b677-84db3008b50e,2K_or_10P,True
...,...,...,...,...
475139,torresfloriberto43@gmail.com,a991f055-28af-4f42-85c6-800704404040,2K_or_10P,True
475140,cristinarzl2107@gmail.com,9bd8614b-cf58-40bd-9d77-1fc125716686,2K,True
475141,ruizeumir_76@yahoo.com,df35a0e6-dcde-468c-9e3d-d701c55a9909,10P,False
475142,arthurbeckham.2099@gmail.com,b8cab4f7-bf09-45b9-9328-e51887cb4b51,2K_or_10P,True


In [11]:
# Cohorts segments
cohorts_info_clean.cashback_type.value_counts()

2K_or_10P    190045
2K           189980
10P           95099
Name: cashback_type, dtype: int64

### Opt-in Info

In [12]:
opt_in_info = pd.read_csv('Opt-In.csv')

In [13]:
opt_in_info.head(10)

,Submission Date,email,source,Nombre,Apellido,IP
0,2022-10-17 10:06:23,cacaxtlso@gmail.com,b908f419-1731-4efe-bdc3-2e197090f775,ddd,scho,187.190.154.251
1,2022-10-22 15:44:13,albertstrong007@hotmail.com,5177f6bc-790b-44c0-9fca-020a7f220cb1,Luis Alberto,Ramírez Rodríguez,189.159.58.51
2,2022-10-22 15:44:13,victorpezglez1985@gmail.com,b9e75169-9c1e-42d7-97d6-8c8e887d31f1,Victor Emmanuel,Perez gonzalez,189.203.205.234
3,2022-10-22 15:44:14,samu17976@gmail.com,66b62548-938b-48bd-b48b-32c34ce60647,Samuel,Gomez,189.217.216.200
4,2022-10-22 15:44:16,black_day696@hotmail.com,e97e2eaf-f498-4c87-9b01-0f5107ae7ad5,GUSTAVO,CORSI BAUTISTA,187.135.158.185
5,2022-10-22 15:44:22,thelmi88@hotmail.com,406bfc41-fbf7-4c06-8414-1a3747cbbb04,Carmela,Calderón Jiménez,189.188.50.18
6,2022-10-22 15:44:29,diazavalos@msn.com,b39a6d84-0321-4f44-97d8-c9757204b8be,Claudia,Díaz Avalos,189.172.152.161
7,2022-10-22 15:44:30,alfsova28@gmail.com,4b388626-4488-4d41-9292-9bb4e2426bb3,Alfonso,Soto,187.131.24.142
8,2022-10-22 15:44:32,topsjavi@gmail.com,1007836b-1cac-459c-aaed-609079be6819,Francisco Javier,Flores Castro,189.145.142.234
9,2022-10-22 15:44:36,reyessantiago220@gmail.com,12d1d84c-95a7-4210-ae91-4a9317b722e3,Agustín,Santiago Reyes,200.63.40.52


In [14]:
# EDA optIn
opt_in_info = opt_in_info[['source']]
opt_in_info.columns = ['user_id']
opt_in_info = pd.DataFrame(opt_in_info.user_id.drop_duplicates(), columns = ['user_id'])
opt_in_info['optIn'] = True

In [15]:
opt_in_info.shape

(31221, 2)

# Cashback Calculation

In [16]:
cashback_october = raw_october.copy()

In [17]:
cashback_october.columns

Index(['user_id', 'ArcusAdjustAmount', 'GalileoAdjustAmount',
       'ArcusPurchaseAmount', 'GalileoPurchaseAmount', 'ArcusAdjustNum',
       'GalileoAdjustNum', 'ArcusPurchaseNum', 'GalileoPurchaseNum'],
      dtype='object')

In [18]:
# Total purchases
cashback_october['total_purchases'] = cashback_october.ArcusPurchaseNum + cashback_october.GalileoPurchaseNum
# Total amount Purchases
cashback_october['amount_purch'] = cashback_october.GalileoPurchaseAmount + cashback_october.ArcusPurchaseAmount
# Calculate the right amount of spend for each user
cashback_october['cashback_amount'] = -1 * cashback_october.amount_purch - cashback_october.GalileoAdjustAmount - - cashback_october.ArcusAdjustAmount
# Calculate the cashback
cashback_october['cashback'] = cashback_october.cashback_amount * 0.01
# Top the cashback for max 1000
cashback_october['cashback'] = cashback_october['cashback'].apply(lambda x: 1000 if x > 1000 else round(x, 1))

#### Clean DataFrame

In [19]:
oct_cashback = cashback_october[['user_id', 'amount_purch', 'total_purchases', 'cashback_amount', 'cashback']]

In [20]:
# Add the segment info
oct_cashback = pd.merge(oct_cashback, cohorts_info_clean, on='user_id', how='left')
# Add optIn responses
oct_cashback = pd.merge(oct_cashback, opt_in_info, on='user_id', how='left')
# Filter the less than 1
oct_cashback = oct_cashback[oct_cashback['cashback'] >= 1]

In [21]:
# Fill users not in experiment
oct_cashback.cashback_type = oct_cashback.cashback_type.fillna('10P')
# Fill users with no required OptIn
oct_cashback.needed_opt_in = oct_cashback.needed_opt_in.fillna(False)
# Fill users with no OptIn
oct_cashback.optIn = oct_cashback.optIn.fillna(False)

/var/folders/b_/lql2bh7s2797fh2q6lw2ttdh0000gn/T/ipykernel_29786/227707460.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  oct_cashback.Segment = oct_cashback.cashback_type.fillna('10P')


In [22]:
oct_cashback.head(10)

,user_id,amount_purch,total_purchases,cashback_amount,cashback,email,cashback_type,needed_opt_in,optIn
0,000048a5-6eb8-464a-ba04-9860337841d2,-300.00,3.0,300.00,3.0,lizgaliher@gmail.com,2K,True,False
1,00007e60-e21e-4616-81b4-4afae84713ee,-739.00,6.0,739.00,7.4,maurochrivas@gmail.com,2K_or_10P,True,False
2,000096ca-40b4-4de0-8b71-6f16939af17f,-308.00,4.0,308.00,3.1,marychuy1971@hotmail.com,10P,False,False
3,0000a054-8132-416b-8cf1-d4c6443bf8ce,-2716.90,8.0,2716.90,27.2,frineegarciamendez@gmail.com,2K_or_10P,True,False
7,0000f477-d13d-4acb-a6c4-b30e6145ba03,-2077.88,13.0,2077.88,20.8,osvalfermg@gmail.com,2K_or_10P,True,True
9,0001fec2-2153-4ba8-a510-c5be80c8c973,-128.00,2.0,128.00,1.3,samuelmolina1a@gmail.com,2K,True,True
10,0002325b-5c34-4e23-89d4-e289b52b1ab7,-135.65,1.0,135.65,1.4,monsesangster@gmail.com,2K,True,False
11,00024ed7-05c1-4421-ae51-0a4d1cd4ecd0,-200.00,2.0,200.00,2.0,arq.jandresve.2019@gmail.com,2K,True,False
12,0002f190-5527-4220-9b92-955bfc498869,-250.00,3.0,250.00,2.5,kaja.sofi15@gmail.com,2K,True,False
13,0002f1c2-58f4-4870-99c6-d3a5fac8c939,-278.00,2.0,278.00,2.8,jareth.rortega10@gmail.com,10P,False,False


### Payment filter

In [23]:
# Filter for payment
oct_cashback['Payment_filter'] = oct_cashback.apply(lambda x:
                                                    ( ( (True if x.cashback_amount >= 2000 else False) if x.cashback_type == '2K' else ( (True if x.cashback_amount >= 2000 or x.total_purchases > 9 else False) if x.cashback_type == '2K_or_10P' else (True if x.total_purchases > 9 else False) ) )
                                                      if x.optIn else False)
                                                    if x.needed_opt_in else
                                                    ( (True if x.cashback_amount >= 2000 else False) if x.cashback_type == '2K' else ( (True if x.cashback_amount >= 2000 or x.total_purchases > 9 else False) if x.cashback_type == '2K_or_10P' else (True if x.total_purchases > 9 else False) ) )
                                                    , axis=1)


In [24]:
oct_cashback

,user_id,amount_purch,total_purchases,cashback_amount,cashback,email,cashback_type,needed_opt_in,optIn,Payment_filter
0,000048a5-6eb8-464a-ba04-9860337841d2,-300.00,3.0,300.00,3.0,lizgaliher@gmail.com,2K,True,False,False
1,00007e60-e21e-4616-81b4-4afae84713ee,-739.00,6.0,739.00,7.4,maurochrivas@gmail.com,2K_or_10P,True,False,False
2,000096ca-40b4-4de0-8b71-6f16939af17f,-308.00,4.0,308.00,3.1,marychuy1971@hotmail.com,10P,False,False,False
3,0000a054-8132-416b-8cf1-d4c6443bf8ce,-2716.90,8.0,2716.90,27.2,frineegarciamendez@gmail.com,2K_or_10P,True,False,False
7,0000f477-d13d-4acb-a6c4-b30e6145ba03,-2077.88,13.0,2077.88,20.8,osvalfermg@gmail.com,2K_or_10P,True,True,True
...,...,...,...,...,...,...,...,...,...,...
197754,fffd510d-47ee-4fff-b20c-bf7712ee1b3d,-1503.78,9.0,1503.78,15.0,mompala90@gmail.com,2K_or_10P,True,False,False
197755,fffe3b68-aae4-4669-bc01-8de21c1a8184,-1745.90,6.0,1745.90,17.5,angelandroabarca@gmail.com,2K,True,False,False
197756,fffe9ec7-6879-4a34-b9db-14c86223830d,-699.00,1.0,699.00,7.0,NaN,NaN,False,False,False
197757,ffff5229-917d-4595-9bb5-8283d4ab8d7b,-9539.49,3.0,9539.49,95.4,euni2199martinez@gmail.com,10P,False,False,False


### Cashback to pay

In [25]:
oct_cashback[oct_cashback.Payment_filter]['cashback'].sum()

1008378.8999999999

# DB Export

In [27]:
oct_cashback.drop('email', axis=1).to_csv('../Cashback_Payments/October.csv')

In [28]:
oct_cashback.shape

(162032, 10)